In [2]:
from openai import OpenAI
from elasticsearch import Elasticsearch
import minsearch, json, requests
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Practice

## Loading dataset

In [ ]:
docs_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1"
docs_response = requests.get(docs_url)
docs_json = docs_response.json()

In [ ]:
documents = []

for course in docs_json:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        

## Minsearch

In [ ]:
min_index = minsearch.Index(
    text_fields=['section', 'question', 'text'],
    keyword_fields=['course']
)

In [ ]:
min_index.fit(documents)

In [ ]:
boost = {'question': 3.0, 'text': 0.5}

retrieved = min_index.search(
    query=query,
    boost_dict=boost,
    filter_dict={'course':'data-engineering-zoomcamp'},
    num_results=5
)

## Elasticsearch

In [3]:
elastic_client = Elasticsearch('http://localhost:9200')

### Building index if not exist

In [ ]:
def build_index(documents):
    index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
            }
        }
    }

    elastic_client.indices.create(index="course-questions", body=index_settings)

    for doc in tqdm(documents):
        elastic_client.index(index="course-questions", document=doc)

In [ ]:
build_index(documents)

### Main

In [38]:
test_query = "How do I execute a command in a running docker container?"

In [11]:
def retriever(question_input):
    query_settings = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": question_input,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    search_results = elastic_client.search(index="course-questions", body=query_settings)

    doc_results = []

    for doc in search_results['hits']['hits']:
        doc_results.append(doc['_source'])

    return doc_results

In [47]:
def prompt_constructor(question_input, context_input):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """

    context = ""

    for doc in context_input:
        context = context + f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"



    full_prompt = prompt_template.format(question= question_input, context= context)
    
    return full_prompt

In [6]:
def generate_answer(prompt):
    openai_client = OpenAI()
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages= [{'role':'user','content':prompt}]
    )

    return response.choices[0].message.content

In [7]:
def main(question):
    retrieved_docs = retriever(question)
    prompts = prompt_constructor(question, retrieved_docs)
    answer = generate_answer(prompts)

    return answer

In [ ]:
question = input("Your question: ")
print()
print(f"Answer: {main(question)}")

# Homework 1

## Q1

## Q2

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
            }
        }
    }

elastic_client.indices.create(index="course-questions", body=index_settings)

for doc in tqdm(documents):
    elastic_client.index(index="course-questions", document=doc)

## Q3

In [56]:
query = "How do execute a command on a Kubernetes pod?"

query_settings = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            }
        }
    }

search_results = elastic_client.search(index="course-questions", body=query_settings)

print(search_results['hits']['hits'][0]['_score'])

44.50556


## Q4

In [79]:
query = "How do copy a file to a Docker container?"

query_settings = {
"size": 3,
"query": {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^3", "text", "section"],
                "type": "best_fields"
            }
        },
        "filter": {
            "term": {
                "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

search_results = elastic_client.search(index="course-questions", body=query_settings)

doc_results = []

for doc in search_results['hits']['hits']:
    doc_results.append(doc['_source'])

print(doc_results[-1]['question'])

How do I copy files from a different folder into docker container’s working directory?


## Q5

In [80]:
query = "How do I execute a command in a running docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context_template = """
Q: {question}
A: {text}
""".strip()

context = ""

for doc in doc_results:
    context = context + context_template.format(question=doc['question'], text=doc['text']) + "\n\n"

full_prompt = prompt_template.format(question= query, context= context)

print(len(full_prompt))

1464


## Q6

In [67]:
import tiktoken

In [81]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [82]:
len(encoding.encode(full_prompt))

323

In [71]:
encoding.decode_single_token_bytes(63842)

b"You're"